# GLTF 格式教學 工具包

In [0]:
class glTF_tools(object):
    def __init__(self):
        return


In [0]:
import json

@staticmethod
def glb_loader(glb_bytes):
    assert glb_bytes[0:4].decode() == "glTF"

    gltf_length = 0

    for idx, curr in enumerate(glb_bytes[8:12]):
        gltf_length += curr * 256 ** idx

    assert len(glb_bytes) == gltf_length

    def chunk_parser(chunk_type, chunk_data):
        return {
            "BIN\u0000": lambda:chunk_data,
            "JSON": lambda:json.loads(chunk_data.decode()),
        }[chunk_type]()

    chunks = []
    offset = 12
    i = 0

    while(offset < len(glb_bytes)):
        chunks.append({ "type": "", "length": 0, "data": "" })

        for idx, curr in enumerate(glb_bytes[offset:offset + 4]):
            chunks[i]["length"] += curr * 256 ** idx

        chunks[i]["type"] = glb_bytes[offset + 4:offset + 8].decode()

        chunks[i]["data"] = chunk_parser(
            chunks[i]["type"],
            glb_bytes[offset + 8:offset + 8 + chunks[i]["length"]]
        )
        offset += 8 + chunks[i]["length"]
        i += 1

    model = chunks[0]["data"]

    for idx, buffer in enumerate(model['buffers']):
        buffer["data"] = chunks[idx + 1]["data"]

    return model

glTF_tools.glb_loader=glb_loader

del glb_loader

In [0]:
import uuid
from IPython.display import display_javascript, display_html

class render_JSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="width:100%;"></div>'.format(self.uuid), raw=True)
        display_html("""
        <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
        <script>
            document.getElementById('%s').appendChild(renderjson(%s))
        </script>
        """ % (self.uuid, self.json_str), raw=True)

glTF_tools.render_JSON = render_JSON

del render_JSON

In [0]:
# !wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb

# glb_file = open('./cube.glb', 'rb')

# glb_bytes = glb_file.read()

In [0]:
# model = glTF_tools.glb_loader(glb_bytes)

In [0]:
# glTF_tools.render_JSON(model)

In [0]:
print("glTF_tools is loaded")